特徴ベクトルに、Word2Vecの平均(名詞のみ抽出)を用いた場合

In [1]:
from keras.layers          import Lambda, Input, Dense, GRU, LSTM, Dropout
from keras.models          import Model,Sequential
from keras.layers.wrappers import Bidirectional
from keras.callbacks       import LambdaCallback 
from keras.optimizers      import Adam
from keras.layers.normalization import BatchNormalization as BN
import keras.backend as K
import random
import sys
import pickle
import glob
import copy
import os
import re
import MeCab
import math
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from gensim.models import word2vec
import numpy as np
from keras.utils import plot_model
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
model = word2vec.Word2Vec.load('/mnt/sdc/wikipedia_data/jawiki_wakati.model')

/home/hikaru/.pyenv/versions/anaconda3-4.4.0/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [8]:
categories=6
addDict={}

In [9]:
def extractKeyword(text,word_class):
    tagger = MeCab.Tagger('-Ochasen')
    tagger.parse('') # <= 空文字列をparseする
    node = tagger.parseToNode(text)
    keywords = []
    while node:
        if node.feature.split(",")[0] == word_class:
            keywords.append(node.surface)
        node = node.next
    return keywords

In [10]:
def predictVector(word, around_words_list):
    global addDict
    if word in addDict:
        return addDict[word]
    else:
        return addUnknownWord(word,around_words_list)

def addUnknownWord(word , around_words_list):
    global addDict
    rand_vector=np.random.rand(200)/np.linalg.norm(np.random.rand(200))*(10+ 3*np.random.rand(1))
    vector=np.array(model[model.predict_output_word(around_words_list)[0][0]])+rand_vector
    addDict[word]=vector
    return vector
    
def Wakati(text):
    m = MeCab.Tagger ("-Ochasen -d /usr/lib/mecab/dic/mecab-ipadic-neologd -Owakati")
    result=m.parse(text)
    ws = re.compile(" ")
    words = [word for word in ws.split(result)]
    if words[-1] == u"\n":
        words = words[:-1]
    return [word for word in words if word!="「" and word!="」" and word!="、"and word!="。"
            and word!="!" and word!="?"]

def seq2vecs(words,predict):
    global addDict
    vectors=[]
    for i in range(len(words)):
            try:
                vectors.append(model[words[i]])
            except:
                if predict:
                    try:
                        vectors.append(predictVector(words[i],[words[i-1]]))
                    except:
                        if i==0:
                            continue
                        else:
                            similar_word=model.similar_by_vector(addDict[words[i-1]], topn=10, restrict_vocab=None)[0][0]
                            vectors.append(predictVector(words[i],[similar_word]))
                else:
                    return []
    return vectors

def train_test_divide(X,Y,test_rate):
    datanum=len(X)
    n=math.floor(datanum*test_rate)
    X_train=np.array(X[:datanum-n])
    Y_train=np.array(Y[:datanum-n])
    X_test=np.array(X[datanum-n:])
    Y_test=np.array(Y[datanum-n:])
    print(X_train.shape)
    print(Y_train.shape)
    print(X_test.shape)
    print(Y_test.shape)
    return (X_train,Y_train),(X_test,Y_test)

def onehot_vector(number):
    global categories
    onehot=np.zeros(categories)
    onehot[number]=1
    return onehot

def load_file(filename):
    ttl=[]
    cat=[]
    with open(filename, "r") as f:
        lines = [line for line in f]
        for line in lines:
            title,category=line.split(" ")
            ttl.append(title)
            cat.append(int(category))
    ttl,cat=shuffle(ttl,cat)
    ttl=np.array(ttl)
    cat=np.array(cat)
    print(ttl.shape)
    print(cat.shape)
    return ttl,cat

def create_data(ttl,cat,predict,sfl):
    X=[]
    Y=[]
    T=[]
    sum=0
    max_len=0
    min_len=1000000
    for i in range(len(ttl)):
        title=ttl[i]
        category=cat[i]
        words=extractKeyword(title,"名詞")
        input_vectors=seq2vecs(words,predict)
        sum+=len(input_vectors)
        max_len=max(max_len,len(input_vectors))
        if len(input_vectors)==0:
            continue
        min_len=min(min_len,len(input_vectors))
        if sfl:
            random.shuffle(input_vectors)
        x=list(np.average(input_vectors,axis=0))    
        y=onehot_vector(int(category))
        X.append(np.array(x))
        Y.append(np.array(y))
        T.append(title)
    X=np.array(X)
    Y=np.array(Y)
    print("-------DataShape------")
    print(X.shape)
    print(Y.shape)
    print(len(T))
    print("-------DataProperties------")
    print("max:"+str(max_len))
    print("min:"+str(min_len))
    print("mean:"+str(sum/len(T)))
    return X,Y,T
def load_dataset(filename,sfl):
    ttl,cat=load_file(filename)
    if filename=="./data/livedoor_data.txt":
        print("file: livedoor")
        X_test,Y_test,T_test=create_data(ttl,cat,predict=True,sfl=False)
        return X_test,Y_test,T_test
    else:
        print("file: Yahoo")
        (train_ttl,train_cat),(test_ttl,test_cat)=train_test_divide(ttl,cat,0.1)
        X_train,Y_train,T_train=create_data(train_ttl,train_cat,predict=False,sfl=False)
        X_test,Y_test,T_test=create_data(test_ttl,test_cat,predict=True,sfl=False)
        return (X_train,Y_train,T_train),(X_test,Y_test,T_test)
    
(X_train,Y_train,T_train),(X_test,Y_test,T_test)=load_dataset("./data/yahoo_data.txt"
                                                              ,sfl=False)    

(83999,)
(83999,)
file: Yahoo
(75600,)
(75600,)
(8399,)
(8399,)


/home/hikaru/.pyenv/versions/anaconda3-4.4.0/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


-------DataShape------
(68817, 200)
(68817, 6)
68817
-------DataProperties------
max:13
min:1
mean:5.3839022334597555


/home/hikaru/.pyenv/versions/anaconda3-4.4.0/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()
/home/hikaru/.pyenv/versions/anaconda3-4.4.0/envs/tensorflow/lib/python3.6/site-packages/gensim/models/word2vec.py:1463: UserWarning: All the input context words are out-of-vocabulary for the current model.
  warnings.warn("All the input context words are out-of-vocabulary for the current model.")
/home/hikaru/.pyenv/versions/anaconda3-4.4.0/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:39: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).


-------DataShape------
(8395, 200)
(8395, 6)
8395
-------DataProperties------
max:12
min:1
mean:5.345205479452055


In [11]:
#モデル作成
model = Sequential()
model.add(Dense(128, input_dim=200,activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(categories, activation='softmax'))
model.summary()
model.compile(optimizer=Adam(), loss='categorical_crossentropy',metrics=["accuracy"])
#モデル学習
history =model.fit(X_train,Y_train,shuffle=True,batch_size=30,epochs=3,
                   validation_split=0.1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               25728     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 390       
Total params: 34,374
Trainable params: 34,374
Non-trainable params: 0
_________________________________________________________________
Train on 61935 samples, validate on 6882 samples
Epoch 1/3
61935/61935 [==============================] - 15s 238us/step - loss: 0.6084 - acc: 0.7876 - val_loss: 0.5532 - val_acc: 0.8053
Epoch 2/3
61935/61935 [==============================] - 11s 181us/step - loss: 0.5029 - acc: 0.8233 - val_loss: 0.5326 - val_acc: 0.8172
Epoch 3/3
61935/61935 [==============================] - 11s 182us/step - los

In [12]:
#plot_history(history)

NameError: name 'plot_history' is not defined

In [13]:
loss_and_metrics = model.evaluate(X_test,Y_test)
print("\nloss:{} accuracy:{}".format(loss_and_metrics[0],loss_and_metrics[1]))

8395/8395 [==============================] - 1s 63us/step

loss:0.5467719480776942 accuracy:0.819178082262781
